In [11]:
# Import statements
import os
import pandas as pd

In [ ]:
# This is taken from the base solution - THIS IS NOT MY WORK 

def read_texts_from_dir(dir_path):
  """
  Reads the texts from a given directory and saves them in the pd.DataFrame with columns ['id', 'file_1', 'file_2'].

  Params:
    dir_path (str): path to the directory with data
  """
  # Count number of directories in the provided path
  dir_count = sum(os.path.isdir(os.path.join(root, d)) for root, dirs, _ in os.walk(dir_path) for d in dirs)
  data=[0 for _ in range(dir_count)]
  print(f"Number of directories: {dir_count}")

  # For each directory, read both file_1.txt and file_2.txt and save results to the list
  i=0
  for folder_name in sorted(os.listdir(dir_path)):
    folder_path = os.path.join(dir_path, folder_name)
    if os.path.isdir(folder_path):
      try:
        with open(os.path.join(folder_path, 'file_1.txt'), 'r', encoding='utf-8') as f1:
          text1 = f1.read().strip()
        with open(os.path.join(folder_path, 'file_2.txt'), 'r', encoding='utf-8') as f2:
          text2 = f2.read().strip()
        index = int(folder_name[-4:])
        data[i]=(index, text1, text2)
        i+=1
      except Exception as e:
        print(f"Error reading directory {folder_name}: {e}")

  # Change list with results into pandas DataFrame
  df = pd.DataFrame(data, columns=['id', 'file_1', 'file_2']).set_index('id')
  return df

In [ ]:
train_dir = "data/train/"
data = []

i = 0 
for dirpath, dirnames, filenames in sorted(os.walk(train_dir)): 
    if i == 0:
        i += 1
        continue 

    try:
        with open(os.path.join(dirpath, filenames[0]), 'r', encoding='utf-8') as f1:
            text1 = f1.read().strip()
        with open(os.path.join(dirpath, filenames[1]), 'r', encoding='utf-8') as f2:
            text2 = f2.read().strip()
        data.append((text1, text2))
    except Exception as e:
        print(f"Error reading directory {dirpath}: {e}")
    
    i += 1 

train_df = pd.DataFrame(data, columns=['file_1', 'file_2'])
train_df.head()

,file_1,file_2
0,The VIRSA (Visible Infrared Survey Telescope A...,The China relay network has released a signifi...
1,China\nThe goal of this project involves achie...,The project aims to achieve an accuracy level ...
2,Scientists can learn about how galaxies form a...,Dinosaur eggshells offer clues about what dino...
3,China\nThe study suggests that multiple star s...,The importance for understanding how stars evo...
4,Dinosaur Rex was excited about his new toy set...,Analyzing how fast stars rotate within a galax...
